In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('data', one_hot=True)

encode_dims = 2
enc_hidden = [1000,1000,encode_dims]
dec_hidden = [1000,1000,784]
disc_hidden = [1000,1000,1]
#enc_hidden = [200,200,200,encode_dims]
#dec_hidden = [200,200,200,784]
#disc_hidden = [200,200,200,1]
#enc_hidden = [100,encode_dims]
#dec_hidden = [100,784]
#disc_hidden = [100,1]

def createHiddenLayers(layers, x, prefix, last_layer_relu):
	trainables = []
	weights_and_biases = []
	next_input_count = x.get_shape()[1].value
	inputs = x
	for idx in range(len(layers)):
		print idx
		hidden_count = layers[idx]
		w_xh = tf.get_variable(prefix + 'w_' + repr(idx), [next_input_count, hidden_count],
			initializer=tf.contrib.layers.xavier_initializer())
#		w_xh = tf.get_variable(prefix + 'w_' + repr(idx), [next_input_count, hidden_count],
#			initializer=tf.truncated_normal_initializer(stddev=0.1))
		if (idx != len(layers) - 1) or (prefix == 'dec_'):
			b_xh = tf.get_variable(prefix + 'b_' + repr(idx), initializer=tf.constant(0.01, shape=[hidden_count]))
		else:
			b_xh = tf.get_variable(prefix + 'b_' + repr(idx), initializer=tf.constant(0.0, shape=[hidden_count]))
 		a_h = tf.add(tf.matmul(inputs, w_xh), b_xh)
		if last_layer_relu or (idx != len(layers) - 1):
			print 'relu'
			y_h = tf.nn.relu(a_h)
		else:
			print 'not relu'
			y_h = a_h
		trainables.append(w_xh)
		trainables.append(b_xh)
		print w_xh.get_shape()
		print b_xh.get_shape()
		weights_and_biases.append([w_xh, b_xh])
		next_input_count = hidden_count
		inputs = y_h
	return y_h, trainables, weights_and_biases

def createNetworkFromWeightsAndBiases(x, weights_and_biases, last_layer_relu):
	inputs = x
	for idx in range(len(weights_and_biases)):
		print idx
		a_h = tf.add(tf.matmul(inputs, weights_and_biases[idx][0]), weights_and_biases[idx][1])
		if last_layer_relu or (idx != len(weights_and_biases) - 1):
			print 'relu'
			y_h = tf.nn.relu(a_h)
		else:
			print 'not relu'
			y_h = a_h
		inputs = y_h
	return y_h

print 'generator: encoder'
gen_enc_x = tf.placeholder(tf.float32, [None, 784])
gen_enc_y, gen_enc_trainables, gen_enc_weights_and_biases = createHiddenLayers(enc_hidden, gen_enc_x, "enc_", False)

print 'generator: encoder'
gen_enc_x2 = tf.placeholder(tf.float32, [None, 784])
gen_enc_y2 =createNetworkFromWeightsAndBiases(gen_enc_x2, gen_enc_weights_and_biases, False)

print 'generator: decoder'
gen_dec_y, gen_dec_trainables, gen_dec_weights_and_biases = createHiddenLayers(dec_hidden, gen_enc_y, "dec_", False)

for x in range(len(gen_dec_weights_and_biases)):
	print gen_dec_weights_and_biases[x][0].get_shape()
	print gen_dec_weights_and_biases[x][1].get_shape()

print 'generator: decoder only'
gen_dec_x2 = tf.placeholder(tf.float32, [None, encode_dims])
gen_dec_y2 = createNetworkFromWeightsAndBiases(gen_dec_x2, gen_dec_weights_and_biases, False)

print 'discriminator - unlinked'
disc_x = tf.placeholder(tf.float32, [None, encode_dims])
disc_a, disc_trainables, disc_weights_and_baises = createHiddenLayers(disc_hidden, disc_x, "disc_", False)
disc_y = tf.sigmoid(disc_a)

print 'discriminator -linked to gen_enc_y2 (z)'
disc_x2 = gen_enc_y2
disc_a2 = createNetworkFromWeightsAndBiases(disc_x2, disc_weights_and_baises, False)
disc_y2 = tf.sigmoid(disc_a2)

y_ = tf.placeholder(tf.float32, [None, 10])
z = tf.placeholder(tf.float32, [None, 2])
z_ = tf.placeholder(tf.float32, [None, 2])
disc_disc_y_ = tf.placeholder(tf.float32, [None, 1])
disc_gen_y_ = tf.placeholder(tf.float32, [None, 1])

#gen_loss = tf.reduce_mean(
#tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=gen_dec_y))

learning_rate = tf.Variable(0.0, trainable=False)

gen_loss = tf.contrib.losses.mean_squared_error(gen_enc_x, gen_dec_y)
#gen_train_step = tf.train.GradientDescentOptimizer(0.1).minimize(gen_loss)
gen_train_step = tf.train.AdamOptimizer(learning_rate=1e-4, beta1=0.5).minimize(gen_loss)

#disc_disc_loss = tf.contrib.losses.log_loss(disc_y, disc_disc_y_)
disc_disc_loss = tf.contrib.losses.mean_squared_error(disc_y, disc_disc_y_)
#disc_disc_loss = tf.contrib.losses.absolute_difference(disc_y, disc_disc_y_)
disc_disc_train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(disc_disc_loss, var_list=disc_trainables)
#disc_disc_train_step = tf.train.AdamOptimizer(learning_rate=1e-4, beta1=0.5).minimize(disc_disc_loss, var_list=disc_trainables)

#disc_enc_loss = tf.contrib.losses.log_loss(disc_y2, disc_gen_y_)
#disc_enc_loss = tf.contrib.losses.mean_squared_error(disc_y2, disc_gen_y_)
disc_enc_loss = tf.contrib.losses.absolute_difference(disc_y2, disc_gen_y_)
disc_enc_train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(disc_enc_loss, var_list=gen_enc_trainables)
#disc_enc_train_step = tf.train.AdamOptimizer(learning_rate=1e-4, beta1=0.5).minimize(disc_enc_loss, var_list=gen_enc_trainables)

jpeg_data = tf.placeholder(tf.float32)
get_constant_mul = tf.fill([28,28,1], 400.0)
get_constant_max = tf.fill([28,28,1], 255.0)
get_constant_min = tf.fill([28,28,1], 0.0)
encode_jpeg = tf.image.encode_jpeg(tf.cast(tf.minimum(tf.maximum(tf.mul(tf.reshape(jpeg_data, [28,28,1]), get_constant_mul), get_constant_min), get_constant_max), tf.uint8), format='grayscale', quality=100)

jpeg_data_grid = tf.placeholder(tf.float32)
get_constant_mul_grid = tf.fill([28*21,28*21,1], 400.0)
get_constant_max_grid = tf.fill([28*21,28*21,1], 255.0)
get_constant_min_grid = tf.fill([28*21,28*21,1], 0.0)
encode_jpeg_grid = tf.image.encode_jpeg(tf.cast(tf.minimum(tf.maximum(tf.mul(tf.reshape(jpeg_data_grid, [28*21,28*21,1]), get_constant_mul_grid), get_constant_min_grid), get_constant_max_grid), tf.uint8), format='grayscale', quality=100)

batch_size = 200

fuzzy_ones_zeros = np.tile(np.array([0.95]), (batch_size,1))
fuzzy_zeros_ones = np.tile(np.array([0.05]), (batch_size,1))
ones_zeros = np.tile(np.array([1.0]), (batch_size,1))
zeros_ones = np.tile(np.array([0.0]), (batch_size,1))
print ones_zeros.shape
print ones_zeros
print zeros_ones.shape
print zeros_ones

#ones_zeros = np.append(np.ones((batch_size,1),dtype=np.float32), np.zeros((batch_size,1),dtype=np.float32), axis=1)
#zeros_ones = np.append(np.zeros((batch_size,1),dtype=np.float32), np.ones((batch_size,1),dtype=np.float32), axis=1)
disc_train_z_ = np.append(fuzzy_ones_zeros, fuzzy_zeros_ones, axis=0)
print disc_train_z_.shape
print disc_train_z_

y1_values = []
y2_values = []
y3_values = []

sess = tf.Session()
sess.run(tf.initialize_all_variables())

# Pre-train discriminator
#for i in range(1001):
#	batch_xs, batch_ys = mnist.train.next_batch(batch_size)
#
#	gen_loss_value, gen_enc_y_result = sess.run([gen_loss, gen_enc_y], feed_dict={gen_enc_x: batch_xs, y_: batch_ys})
#
#	sampled_z = np.random.uniform(1.0, 1.0, size=(batch_size, encode_dims))
##	sampled_z2 = np.random.uniform(-2.0, -1.0, size=(batch_size, encode_dims))
#	train_z = np.append(sampled_z, gen_enc_y_result, axis=0)
##	train_z = np.append(sampled_z, sampled_z2, axis=0)
##	print train_z

#	_, disc_disc_loss_value, disc_y_out = sess.run([disc_disc_train_step, disc_disc_loss, disc_y], feed_dict={disc_x: train_z, disc_disc_y_: disc_train_z_})
#	_, disc_enc_loss_value = sess.run([disc_enc_train_step, disc_enc_loss], feed_dict={gen_enc_x2: batch_xs, disc_gen_y_: ones_zeros})
#
#	print('{}  {:0.5f}  {:0.5f}  {:0.5f}  {:0.5f}  {:0.5f}  {:0.5f}  {:0.5f}  {:0.5f}'.format(i, gen_loss_value, disc_disc_loss_value, np.mean(gen_enc_y_result[0]), np.mean(gen_enc_y_result[1]), np.min(gen_enc_y_result[0]), np.max(gen_enc_y_result[0]), np.min(gen_enc_y_result[1]), np.max(gen_enc_y_result[1])))

# Train
for i in range(100001):
	if i < 1000:
		sess.run(learning_rate.assign(0.1))
	elif i < 5000:
		sess.run(learning_rate.assign(0.05))
	else:
		sess.run(learning_rate.assign(0.02))

	batch_xs, batch_ys = mnist.train.next_batch(batch_size)
	_, gen_loss_value, gen_enc_y_result = sess.run([gen_train_step, gen_loss, gen_enc_y], feed_dict={gen_enc_x: batch_xs, y_: batch_ys})

	sampled_z = np.random.uniform(-1.0, 1.0, size=(batch_size, encode_dims))
	train_z = np.append(sampled_z, gen_enc_y_result, axis=0)
#	print gen_enc_y_result.shape
#	print sampled_z.shape
#	print train_z.shape
#	print train_z.shape
#	print ones_zeros.shape
#	print zeros_ones.shape
#	print disc_train_z_.shape
#	print sampled_z
#	if i % 5 == 0:
#		plt.gcf().clear()
#		for z_idx in range(len(train_z)):
#			plt.scatter(train_z[z_idx,0], train_z[z_idx,1], color='black', alpha=0.25)
#		plt.savefig('out/test_enc_' + repr(i) + '.png')
#		plt.gcf().clear()

	_, disc_disc_loss_value, disc_y_out = sess.run([disc_disc_train_step, disc_disc_loss, disc_y], feed_dict={disc_x: train_z, disc_disc_y_: disc_train_z_})
	_, disc_enc_loss_value = sess.run([disc_enc_train_step, disc_enc_loss], feed_dict={gen_enc_x2: batch_xs, disc_gen_y_: ones_zeros})

	y1_values.append(gen_loss_value)
	y2_values.append(disc_disc_loss_value)
	y3_values.append(disc_enc_loss_value)

	if i % 100 == 0:
		plt.gcf().clear()
		plt.plot(y1_values, '-r', label='gen_loss_value')
		plt.plot(y2_values, '-g', label='disc_disc_loss_value')
		plt.plot(y3_values, '-b', label='disc_enc_loss_value')
		plt.savefig('out/losses_' + repr(i) + '.png')
		validation_loss, z_out = sess.run([gen_loss, gen_enc_y], feed_dict={gen_enc_x: mnist.test.images[0:500], y_: mnist.test.labels[0:500]})
		
		digit_colors = ['red', 'green', 'blue', 'cyan', 'magenta',
			'yellow', 'black', 'white', 'orange', 'gray',]
		plt.gcf().clear()
		for z_idx in range(len(z_out)):
			color = (mnist.test.labels[z_idx] > 0).nonzero()[0][0]
			plt.scatter(z_out[z_idx,0], z_out[z_idx,1], color=digit_colors[color], alpha=0.25)
		plt.savefig('out/test_scatter_' + repr(i) + '.png')
		plt.gcf().clear()

		for z_idx in range(batch_size):
			color = (batch_ys[z_idx] > 0).nonzero()[0][0]
			plt.scatter(gen_enc_y_result[z_idx,0], gen_enc_y_result[z_idx,1], color=digit_colors[color], alpha=0.25)
		plt.savefig('out/train_scatter_' + repr(i) + '.png')

		# Find 0-9 in the train data, encode and decode and save .jpg of decoded images
		n = 0
		idx = 0
		while n < 10:
			batch_xs = [mnist.test.images[idx]]
			batch_ys = [mnist.test.labels[idx]]
			logit = (mnist.test.labels[idx] > 0).nonzero()[0][0]
			if logit == n:
				output_z, output_y = sess.run([gen_enc_y, gen_dec_y], feed_dict={gen_enc_x: batch_xs, y_: batch_ys})
				print output_z
				output_jpeg = sess.run(encode_jpeg, feed_dict={ jpeg_data: output_y })
				with open('out/test_' + repr(i) + '_' + repr(n) + '.jpg', 'w') as fd:
					fd.write(output_jpeg)
				n = n + 1
			idx = idx + 1
		# output grid of decoded digits across z
		grid = np.array([])
		for y in np.arange(-1.0, 1.1, 0.1):
			for x in np.arange(-1.0, 1.1, 0.1):
				z = np.array([[x,y]])
				output = sess.run([gen_dec_y2], feed_dict={gen_dec_x2: z})
				output = np.resize(output,(28,28))
				if x == -1.0:
					row = output
				else:
					row =  np.concatenate((row, output), axis=1)
			if y == -1.0:
				grid = row
			else:
				grid =  np.concatenate((grid, row), axis=0)
		output_jpeg_grid = sess.run(encode_jpeg_grid, feed_dict={ jpeg_data_grid: grid })
		with open('out/grid_' + repr(i) + '.jpg', 'w') as fd:
			fd.write(output_jpeg_grid)
		# print losses
		print('{}  {:0.5f}  {:0.5f}  {:0.5f}  {:0.5f}  {:0.5f}  {:0.5f}'.format(i, gen_loss_value, disc_disc_loss_value, disc_enc_loss_value, np.mean(gen_enc_y_result[0]), np.mean(gen_enc_y_result[1]), validation_loss))
	else:
		print('{}  {:0.5f}  {:0.5f}  {:0.5f}  {:0.5f}  {:0.5f}'.format(i, gen_loss_value, disc_disc_loss_value, disc_enc_loss_value, np.mean(gen_enc_y_result[0]), np.mean(gen_enc_y_result[1])))



# 0.47
# batch 100->200
# and reduce learning rate for disc over time
# gives 0.42 and really good results!!! all digits recognizable in grid at 10,000 rounds, bit better at 20,000
# batch 200->400
